In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from agents.q_learner import Q_learner
from agents.distrib_learner import Distrib_learner

%matplotlib inline

In [2]:
indices = torch.Tensor([[2],[1],[0]]).long()
print(indices.size())
mask = torch.zeros(3, 3)
src = torch.randn(3,3)
print(src)
print(mask)
mask.scatter_(1, indices , src)
print(mask)


torch.Size([3, 1])
tensor([[ 1.2571, -0.0506, -1.2294],
        [-0.1495,  0.0245, -0.0395],
        [-0.2384, -0.4982,  2.8169]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[ 0.0000,  0.0000,  1.2571],
        [ 0.0000, -0.1495,  0.0000],
        [-0.2384,  0.0000,  0.0000]])


In [3]:
args = dict()
args["BUFFER_SIZE"] = int(500)  # replay buffer size
args["BATCH_SIZE"] = 32  # minibatch size
args["GAMMA"] = 0.95  # discount factor
args["TAU"] = 1e-3  # for soft update of target parameters
args["LR"] = 0.001  # learning rate
args["UPDATE_EVERY"] = 4  # how often to update the network

In [4]:
env = gym.make('CartPole-v1')
env.seed(0)
agent = Distrib_learner(N=100, Vmin=-300, Vmax=300, state_size=env.observation_space.shape[0], action_size= env.action_space.n, seed=0, hiddens = [24,24], args = args)

In [5]:
state = env.reset()

In [6]:
def dqn(n_episodes=10000, max_t=1000, eps_start=1, eps_end=0.01, eps_decay=0.995):

    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state.reshape([1, -1]), eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'models/checkpoints/checkpoint.pth')
            break
    return scores

scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)